In [1]:
from tools import get_image_dimensions

c:\Users\dell\anaconda3\envs\ssg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
get_image_dimensions('img_p0_1.png')

(722, 406, 722, 406)

In [7]:
# outline_generator.py
import os
import json
from typing import List
from pydantic import BaseModel, ValidationError
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Pydantic models for validation
class SectionOutline(BaseModel):
    heading: str
    num_content_slides: int
    key_points: List[str]

class DocumentOutline(BaseModel):
    title: str
    subtitle: str | None = None  # Make subtitle optional with default None
    sections: List[SectionOutline]

# Prompt template
PROMPT_TEMPLATE = """
You are a slide-outline assistant.  
Given the document title and a list of section summaries, produce a JSON object matching this schema:

{{
  "title": <string>,           // main deck title
  "subtitle": <string>,        // optional subtitle or author
  "sections": [
    {{
      "heading": <string>,     // section heading
      "num_content_slides": <integer >= 1>,  
      "key_points": [<string>, …]   // 3–5 bullets that will become slide bullets
    }},
    …
  ]
}}

Use no extra keys. Constrain total slides (sum of num_content_slides) to be at most {max_slides}.  
Here are the inputs:

Document Title:
{title}

Section Summaries:
{summaries}
"""

def generate_outline(title: str, summaries: List[dict], max_slides: int = 15) -> DocumentOutline:
    # Fill prompt
    prompt = PROMPT_TEMPLATE.format(
        title=title,
        summaries="\n".join(f"- {s['title']}: {s['summary']}" for s in summaries),  # Include section titles
        max_slides=max_slides
    )

    # Call the LLM
    resp = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=1000,  # Increased token limit for longer responses
    )

    # Parse JSON
    content = resp.choices[0].message.content.strip()
    try:
        outline_dict = json.loads(content)
    except json.JSONDecodeError as e:
        raise RuntimeError(f"LLM returned invalid JSON:\n{content}") from e

    # Validate with Pydantic
    try:
        outline = DocumentOutline.model_validate(outline_dict)
    except ValidationError as e:
        raise RuntimeError(f"Outline validation failed:\n{e}") from e

    return outline

def pretty_print(outline: DocumentOutline):
    print(f"\nDeck Title: {outline.title}")
    if outline.subtitle:
        print(f"Subtitle: {outline.subtitle}\n")
    total = 0
    for sec in outline.sections:
        print(f"Section: {sec.heading}  •  Slides: {sec.num_content_slides}")
        for pt in sec.key_points:
            print(f"   – {pt}")
        total += sec.num_content_slides
        print()
    print(f"Total slides planned: {total}\n")

if __name__ == "__main__":
    # Example usage; replace with your real summaries
    doc_title = "Q2 Marketing Strategy Review"
    section_summaries = [{'title': 'Brief Assessment:', 'summary': 'Yue Sai, a once prestigious Chinese cosmetics brand, lost relevance due to failed repositionings, leading to low awareness among younger consumers. L’Oréal faces the challenge of reviving the brand in a competitive market shaped by rising local players and digital consumption. Younger Chinese consumers prefer skin care rooted in tradition. Competitors like Herborist succeed with modern branding rooted in traditional Chinese medicine. Digital platforms and experiential retail offer better engagement than TV ads. L’Oréal must align its city-tier strategy with a clear brand identity to meet evolving consumer expectations.'}, {'title': 'Decision Problem:', 'summary': "To reposition Yue Sai in China's cosmetics market, L'Oréal must conduct thorough market research to understand current trends and consumer preferences. By leveraging Yue Sai's heritage and reputation, L'Oréal can revamp the brand's image to appeal to modern Chinese consumers. Implementing innovative marketing strategies and product offerings tailored to the local market will be crucial in regaining relevance and increasing market share."}, {'title': 'Criteria:', 'summary': "This slide examines Yue Sai's sales performance and product-market fit. Sales data is analyzed to project revenue, while market research assesses how well products align with target segment needs. Factors like local relevance and demographic responsiveness are considered to ensure products meet customer preferences."}, {'title': '3. Strategic Fit', 'summary': "This slide evaluates the alignment of alternatives with L'Oréal's brand vision and mission. Each alternative is scored based on its compatibility with the company's brand values."}, {'title': '4. Risk of Attrition', 'summary': "Learn how to measure customer loyalty and growth with a focus on retaining existing customers and attracting new ones. Evaluate customer retention metrics, purchase behavior, and changes in demographics to ensure the product's success in the market."}, {'title': '5. Execution Complexity', 'summary': 'This slide evaluates the implementation difficulty of the proposed alternative by assessing costs related to resources, budget constraints, production, and logistics. It highlights the importance of considering these factors when making decisions.'}, {'title': 'Moderate Risk', 'summary': 'In a highly competitive digital space, the brand risks alienating older loyalists while facing strong local rivals like Herborist and Chando.'}, {'title': 'Attrition Risk', 'summary': 'Appealing to a broader audience can lead to increased retention rates, but may also dilute brand prestige.'}, {'title': 'Execution', 'summary': 'To achieve moderate growth, implement targeted digital campaigns, invest in TCM research and development, and expand offerings in premium tiers.'}, {'title': 'Very High', 'summary': 'Managing multiple channels can significantly raise complexity and costs.'}, {'title': 'Recommendation: Alternative 1 - Yue Sai for Modern Young Women', 'summary': "Alternative 1 offers a clear strategy for Yue Sai to rebuild its brand with luxury appeal and cultural relevance through TCM, targeting modern, health-conscious young women in China. This unique position within L'Oreal's portfolio distinguishes Yue Sai from other luxury brands, ensuring long-term growth and market relevance."}]
    outline = generate_outline(doc_title, section_summaries, max_slides=12)
    pretty_print(outline)


Deck Title: Q2 Marketing Strategy Review
Section: Brief Assessment  •  Slides: 2
   – Yue Sai lost relevance due to failed repositionings
   – Competitive market with rising local players and digital consumption
   – Align city-tier strategy with clear brand identity

Section: Decision Problem  •  Slides: 2
   – Thorough market research needed to understand trends and preferences
   – Leverage heritage and reputation to appeal to modern consumers
   – Implement innovative marketing strategies tailored to local market

Section: Criteria  •  Slides: 1
   – Analyze sales performance and product-market fit
   – Assess local relevance and demographic responsiveness

Section: Strategic Fit  •  Slides: 1
   – Evaluate alignment of alternatives with brand vision and mission

Section: Risk of Attrition  •  Slides: 1
   – Measure customer loyalty and growth
   – Evaluate customer retention metrics and purchase behavior

Section: Execution Complexity  •  Slides: 1
   – Assess implementation diff

In [11]:
def save_outline(outline: DocumentOutline, file_path: str):
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(outline.model_dump(), f, indent=2, ensure_ascii=False)


In [12]:
save_outline(outline, "outline.json")

In [13]:
def load_outline(file_path: str) -> DocumentOutline:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return DocumentOutline.model_validate(data)


In [14]:
load_outline("outline.json")

DocumentOutline(title='Q2 Marketing Strategy Review', subtitle='', sections=[SectionOutline(heading='Brief Assessment', num_content_slides=2, key_points=['Yue Sai lost relevance due to failed repositionings', 'Competitive market with rising local players and digital consumption', 'Align city-tier strategy with clear brand identity']), SectionOutline(heading='Decision Problem', num_content_slides=2, key_points=['Thorough market research needed to understand trends and preferences', 'Leverage heritage and reputation to appeal to modern consumers', 'Implement innovative marketing strategies tailored to local market']), SectionOutline(heading='Criteria', num_content_slides=1, key_points=['Analyze sales performance and product-market fit', 'Assess local relevance and demographic responsiveness']), SectionOutline(heading='Strategic Fit', num_content_slides=1, key_points=['Evaluate alignment of alternatives with brand vision and mission']), SectionOutline(heading='Risk of Attrition', num_cont

In [20]:
from pydantic import BaseModel
from typing import List, Optional

class SlideContent(BaseModel):
    title: str
    bullets: List[str]
    speaker_notes: str
    image_caption: Optional[str] = None


In [ ]:
def generate_slide_content(section_heading: str, key_point: str, need_image: bool = False) -> SlideContent:
    system_prompt = "You are a presentation writing assistant."
    user_prompt = f"""
Generate content for one PowerPoint slide.

Slide Context:
- Section heading: "{section_heading}"
- Key point: "{key_point}"

Output in this format (no extra commentary):

{{
  "title": <Slide title>,
  "bullets": [<bullet point 1>, <bullet point 2>, ...],
  "speaker_notes": <narration text>,
  "image_caption": <caption describing what kind of image should go on this slide>  # Omit if no image needed
}}

Guidelines:
- Keep bullets short, concise, and professional.
- Speaker notes should be 2-3 sentences expanding on the bullets.
- If `image_caption` is included, describe the image in a way that helps retrieve it from an image search or RAG system.
"""
    resp = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.5,
        max_tokens=600,
    )

    content = resp.choices[0].message.content.strip()
    try:
        slide_dict = json.loads(content)
    except json.JSONDecodeError as e:
        raise RuntimeError(f"Invalid JSON from LLM:\n{content}") from e

    return SlideContent.model_validate(slide_dict)


In [29]:
def generate_deck_slides(outline: DocumentOutline, image_every_n: int = 2) -> List[SlideContent]:
    print('inside generate_deck_slides')
    all_slides = []
    slide_counter = 0

    for section in outline.sections:
        for i, key_point in enumerate(section.key_points):
            print(f"Generating slide {slide_counter + 1} of {len(outline.sections) * len(outline.sections[0].key_points)}")
            need_image = ((slide_counter % image_every_n) == 0)
            slide = generate_slide_content(
                section_heading=section.heading,
                key_point=key_point,
                need_image=need_image
            )
            all_slides.append(slide)
            slide_counter += 1

    return all_slides


In [30]:
print('hi')

hi


In [31]:
generate_deck_slides(outline)

inside generate_deck_slides
Generating slide 1 of 21
Generating slide 2 of 21
Generating slide 3 of 21
Generating slide 4 of 21
Generating slide 5 of 21
Generating slide 6 of 21
Generating slide 7 of 21
Generating slide 8 of 21
Generating slide 9 of 21
Generating slide 10 of 21
Generating slide 11 of 21
Generating slide 12 of 21
Generating slide 13 of 21
Generating slide 14 of 21
Generating slide 15 of 21
Generating slide 16 of 21


[SlideContent(title="Assessment: Yue Sai's Loss of Relevance", bullets=['Multiple failed attempts to reposition the brand', 'Loss of relevance among target consumers', 'Decline in brand value and market share'], speaker_notes='Yue Sai, once a prominent brand, has lost its relevance due to several unsuccessful repositioning attempts. This has led to a disconnect with its target audience, resulting in a significant decrease in brand value and market share.', image_caption="A graph showing the decline in Yue Sai's market share over time"),
 SlideContent(title='Brief Assessment: Market Competition & Digital Consumption', bullets=['Increasing competition from emerging local players', 'Significant growth in digital consumption', 'Necessity for strategic adaptation to stay competitive'], speaker_notes="The market is experiencing a surge in competition due to the rise of local players. Concurrently, there's a notable increase in digital consumption which is reshaping the industry landscape. To

In [ ]:
# Figure out why 21 slides are being generated. 
# There were 7 key points. Not all of them need their own slides. 
